# Análise de dados dos jogos do Flamengo 2024

In [1]:
# Carregando algumas bibliotecas importantes
import numpy as np
import pandas as pd 
import csv
from datetime import datetime

In [2]:
# Importando os arquivos
jogos = pd.read_csv('../data/processed/matches_FLA2024.csv', parse_dates=['kick-off'])
gols = pd.read_csv('../data/processed/goal_scorers_2024.csv')

## Estatísticas 

In [3]:
# Calculando os gols pró, sofridos e saldo de gols
gols_pro = jogos.gols_casa.sum() + jogos.gols_fora.sum()
gols_contra = jogos.gols_sofridos_casa.sum() + jogos.gols_sofridos_fora.sum()
saldo_gols = gols_pro - gols_contra

In [4]:
# Obtendo o número de vitórias, derrotas, empates e total de jogos
vitorias = jogos['resultado'].value_counts()[0]
derrotas = jogos['resultado'].value_counts()[2]
empates = jogos['resultado'].value_counts()[1]
total_jogos = vitorias + empates + derrotas

# Gols pro e sofridos por jogo
gols_per_jogo = round(gols_pro/total_jogos,2)
gols_sofrido_jogo = round(gols_contra/total_jogos,2)

# Calculando o aproveitamento
aproveitamento = round((3*vitorias + 1*empates + 0*derrotas)/(3*total_jogos)*100,2)
aproveitamento_pontos = (3*vitorias + 1*empates + 0*derrotas)

In [5]:
jogos

,id_match,home_team,away_team,home_score,away_score,tournament,kick-off,stadium,attendance,torneio,...,gols_casa,gols_sofridos_casa,gols_fora,gols_sofridos_fora,marcou_gol,sofreu_gol,jogos_cumulativos,year,month,weekday
0,CA1,Flamengo,Audax-RJ,4,0,Carioca - Round 1,2024-01-17 21:30:00,Arena da Amazônia-AM,44068,Carioca,...,4,0,0,0,Sim,Não,1,2024,Jan,Wed
1,AM1,Philadelphia Union-EUA,Flamengo,0,2,International Friendly Game,2024-01-21 16:00:00,AI Lang Stadium-EUA,7227,International Friendly Game,...,0,0,2,0,Sim,Não,2,2024,Jan,Sun
2,CA2,Nova Iguaçu-RJ,Flamengo,1,1,Carioca - Round 2,2024-01-21 18:10:00,Estádio Almeidão-PB,16428,Carioca,...,0,0,1,1,Sim,Sim,3,2024,Jan,Sun
3,AM2,Orlando City,Flamengo,1,1,International Friendly Game,2024-01-27 16:00:00,Inter&Co Stadium-EUA,22678,International Friendly Game,...,0,0,1,1,Sim,Sim,4,2024,Jan,Sat
4,CA3,Portuguesa-RJ,Flamengo,0,0,Carioca - Round 3,2024-01-27 18:10:00,Arena das Dunas-RN,25211,Carioca,...,0,0,0,0,Não,Não,5,2024,Jan,Sat
5,CA4,Sampaio Corrêa-RJ,Flamengo,0,2,Carioca - Round 4,2024-01-31 21:30:00,Mangueirão-PA,47644,Carioca,...,0,0,2,0,Sim,Não,6,2024,Jan,Wed
6,CA5,Vasco,Flamengo,0,0,Carioca - Round 5,2024-02-04 19:00:00,Maracanã-RJ,56318,Carioca,...,0,0,0,0,Não,Não,7,2024,Feb,Sun
7,CA6,Flamengo,Botafogo,1,0,Carioca - Round 6,2024-02-07 21:30:00,Maracanã-RJ,46798,Carioca,...,1,0,0,0,Sim,Não,8,2024,Feb,Wed
8,CA7,Flamengo,Volta Redonda,3,0,Carioca - Round 7,2024-02-10 16:00:00,Maracanã-RJ,42373,Carioca,...,3,0,0,0,Sim,Não,9,2024,Feb,Sat
9,CA8,Bangu-RJ,Flamengo,0,3,Carioca - Round 8,2024-02-15 21:30:00,Arena Batistão-SE,15575,Carioca,...,0,0,3,0,Sim,Não,10,2024,Feb,Thu


In [6]:
jogos_marcou = jogos.marcou_gol.value_counts()[0]
jogos_nao_marcou = jogos.marcou_gol.value_counts()[1]

jogos_sofreu = jogos.sofreu_gol.value_counts(ascending=True)[0]
jogos_nao_sofreu = jogos.sofreu_gol.value_counts(ascending=True)[1]

In [7]:
stats = pd.DataFrame(jogos[['id_match', 'marcou_gol', 'sofreu_gol', 'kick-off', 'resultado', 'locale', 'jogos_cumulativos']])

stats['gols_marcados'] = jogos.gols_casa + jogos.gols_fora
stats['gols_sofridos'] = jogos.gols_sofridos_casa + jogos.gols_sofridos_fora

stats['gols_marcados_acumulados'] = stats['gols_marcados'].cumsum()
stats['gols_sofridos_acumulados'] = stats['gols_sofridos'].cumsum()
stats['diferença_gols_acumulados'] = stats['gols_marcados_acumulados'] - stats['gols_sofridos_acumulados']

In [8]:
print(
"""
Jogos: {0}
Vitórias: {1}
Empates: {2}
Derrotas: {3} 
Gols pro: {4}
Gols contra: {5}
Saldo de gols: +{6}
Gols/jogo: {7}
Gols sofridos/jogo: {8}
Jogos em que marcou: {9}
Jogos em que sofreu gol: {10} 
Aproveitamento: {11}%
Aproveitamento pontos: {12}/{13}
""".format(total_jogos,vitorias,empates,derrotas,gols_pro,gols_contra,saldo_gols,gols_per_jogo,
           gols_sofrido_jogo,jogos_marcou,jogos_sofreu,aproveitamento,aproveitamento_pontos, 3*total_jogos))


Jogos: 59
Vitórias: 34
Empates: 13
Derrotas: 12 
Gols pro: 93
Gols contra: 41
Saldo de gols: +52
Gols/jogo: 1.58
Gols sofridos/jogo: 0.69
Jogos em que marcou: 48
Jogos em que sofreu gol: 29 
Aproveitamento: 64.97%
Aproveitamento pontos: 115/177



### Criando tabelas com o número de gols e assistências

In [9]:
# Fazendo o merge dos dataframes jogos e gols
df = pd.merge(jogos, gols, on = 'id_match')
df.head()

,id_match,home_team,away_team,home_score,away_score,tournament,kick-off,stadium,attendance,torneio,...,gols_fora,gols_sofridos_fora,marcou_gol,sofreu_gol,jogos_cumulativos,year,month,weekday,gols,assist
0,CA1,Flamengo,Audax-RJ,4,0,Carioca - Round 1,2024-01-17 21:30:00,Arena da Amazônia-AM,44068,Carioca,...,0,0,Sim,Não,1,2024,Jan,Wed,Léo Pereira,Arrascaeta
1,CA1,Flamengo,Audax-RJ,4,0,Carioca - Round 1,2024-01-17 21:30:00,Arena da Amazônia-AM,44068,Carioca,...,0,0,Sim,Não,1,2024,Jan,Wed,Pedro,Gerson
2,CA1,Flamengo,Audax-RJ,4,0,Carioca - Round 1,2024-01-17 21:30:00,Arena da Amazônia-AM,44068,Carioca,...,0,0,Sim,Não,1,2024,Jan,Wed,Everton Cebolinha,NaN
3,CA1,Flamengo,Audax-RJ,4,0,Carioca - Round 1,2024-01-17 21:30:00,Arena da Amazônia-AM,44068,Carioca,...,0,0,Sim,Não,1,2024,Jan,Wed,Varela,Luiz Araújo
4,AM1,Philadelphia Union-EUA,Flamengo,0,2,International Friendly Game,2024-01-21 16:00:00,AI Lang Stadium-EUA,7227,International Friendly Game,...,2,0,Sim,Não,2,2024,Jan,Sun,Pedro,NaN


In [10]:
# Quantidade de gols por jogador
df_gols_counts = pd.DataFrame(gols.gols.value_counts())
df_gols_counts = df_gols_counts.reset_index()
df_gols_counts.columns = ['Jogador', 'Gols'] 

# Quantidade de assistência por jogador

df_assist_counts = pd.DataFrame(gols.assist.value_counts())
df_assist_counts = df_assist_counts.reset_index()
df_assist_counts.columns = ['Jogador', 'Assistências'] 

new_df = pd.concat([df_assist_counts, df_gols_counts], axis=1)
print(new_df.to_markdown(index=False))

| Jogador           |   Assistências | Jogador           |   Gols |
|:------------------|---------------:|:------------------|-------:|
| Luiz Araújo       |             10 | Pedro             |     32 |
| Arrascaeta        |              9 | Bruno Henrique    |      8 |
| Pedro             |              8 | Arrascaeta        |      8 |
| Gerson            |              7 | Everton Cebolinha |      6 |
| Ayrton Lucas      |              5 | Luiz Araújo       |      6 |
| de la Cruz        |              5 | Gabi              |      4 |
| Everton Cebolinha |              4 | Léo Pereira       |      3 |
| Lorran            |              3 | Ayrton Lucas      |      3 |
| Igor Jesus        |              2 | Own Goal          |      3 |
| Viña              |              2 | Gerson            |      3 |
| Léo Ortiz         |              2 | Carlinhos         |      2 |
| Varela            |              1 | David Luiz        |      2 |
| Pulgar            |              1 | de la Cru

In [11]:
# Unificando as tabelas gols e assistências
uni_df = pd.merge(df_gols_counts, df_assist_counts, on = 'Jogador', how ="left").fillna(0).head(15)

print(uni_df.to_markdown(index=False))

| Jogador           |   Gols |   Assistências |
|:------------------|-------:|---------------:|
| Pedro             |     32 |              8 |
| Bruno Henrique    |      8 |              1 |
| Arrascaeta        |      8 |              9 |
| Everton Cebolinha |      6 |              4 |
| Luiz Araújo       |      6 |             10 |
| Gabi              |      4 |              1 |
| Léo Pereira       |      3 |              0 |
| Ayrton Lucas      |      3 |              5 |
| Own Goal          |      3 |              0 |
| Gerson            |      3 |              7 |
| Carlinhos         |      2 |              0 |
| David Luiz        |      2 |              0 |
| de la Cruz        |      2 |              5 |
| Viña              |      1 |              2 |
| Lorran            |      1 |              3 |


In [12]:
stats['year'] = stats['kick-off'].map(lambda x: datetime.strftime(x,"%Y"))
stats['month'] = stats['kick-off'].map(lambda x: datetime.strftime(x,"%b"))
stats['weekday'] = stats['kick-off'].map(lambda x: datetime.strftime(x,"%a"))

In [13]:
stats.to_csv('../data/processed/stats_FLA2024.csv', index=False)

In [14]:
stats

,id_match,marcou_gol,sofreu_gol,kick-off,resultado,locale,jogos_cumulativos,gols_marcados,gols_sofridos,gols_marcados_acumulados,gols_sofridos_acumulados,diferença_gols_acumulados,year,month,weekday
0,CA1,Sim,Não,2024-01-17 21:30:00,Vitória,Casa,1,4,0,4,0,4,2024,Jan,Wed
1,AM1,Sim,Não,2024-01-21 16:00:00,Vitória,Fora,2,2,0,6,0,6,2024,Jan,Sun
2,CA2,Sim,Sim,2024-01-21 18:10:00,Empate,Fora,3,1,1,7,1,6,2024,Jan,Sun
3,AM2,Sim,Sim,2024-01-27 16:00:00,Empate,Fora,4,1,1,8,2,6,2024,Jan,Sat
4,CA3,Não,Não,2024-01-27 18:10:00,Empate,Fora,5,0,0,8,2,6,2024,Jan,Sat
5,CA4,Sim,Não,2024-01-31 21:30:00,Vitória,Fora,6,2,0,10,2,8,2024,Jan,Wed
6,CA5,Não,Não,2024-02-04 19:00:00,Empate,Fora,7,0,0,10,2,8,2024,Feb,Sun
7,CA6,Sim,Não,2024-02-07 21:30:00,Vitória,Casa,8,1,0,11,2,9,2024,Feb,Wed
8,CA7,Sim,Não,2024-02-10 16:00:00,Vitória,Casa,9,3,0,14,2,12,2024,Feb,Sat
9,CA8,Sim,Não,2024-02-15 21:30:00,Vitória,Fora,10,3,0,17,2,15,2024,Feb,Thu
